# LLaRA evaluation

## Load all evaluation results

In [1]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

levels = ['placement_generalization',
 'combinatorial_generalization',
 'novel_object_generalization',
 'novel_task_generalization']

files = glob('[*.json')
result = []

for f in files:
    model_name = f[:-5]
    model_name = re.sub(r'\(.*\)', '', model_name).split(']')[-1]
    
    js = json.load(open(f, 'r'))

    # about prompt mode
    pm = ''
    prop = []
    model_path = None
    for i, j in js.items():
        if i == 'global':
            pm = j.get('prompt_mode', 'N/A')
            model_path = j.get('model_path', None)
        else:
            try:
                del j['lm_prompt_hist']
                del j['lm_answer_hist']
            except:
                pass
            prop.append(j)

    for i in prop:
        i['level'] = f"L{levels.index(i['level']) + 1}"
        i['method'] = model_name
        i['prompt_mode'] = pm
    result.extend(prop)


df = pd.DataFrame(result).fillna('')
display(df.head())

,tid,level,task,seed,prompt,step,success,failure,method,prompt_mode
0,placement_generalization/sweep_without_exceedi...,L1,sweep_without_exceeding,200000,Sweep two {swept_obj} into {bounds} without ex...,2,True,False,D-inBC-AuxB-VIMA-80k,hso
1,placement_generalization/sweep_without_exceedi...,L1,sweep_without_exceeding,200001,Sweep any {swept_obj} into {bounds} without ex...,1,True,False,D-inBC-AuxB-VIMA-80k,hso
2,placement_generalization/sweep_without_exceedi...,L1,sweep_without_exceeding,200002,Sweep all {swept_obj} into {bounds} without ex...,2,True,False,D-inBC-AuxB-VIMA-80k,hso
3,placement_generalization/sweep_without_exceedi...,L1,sweep_without_exceeding,200003,Sweep two {swept_obj} into {bounds} without ex...,2,True,False,D-inBC-AuxB-VIMA-80k,hso
4,placement_generalization/sweep_without_exceedi...,L1,sweep_without_exceeding,200004,Sweep two {swept_obj} into {bounds} without ex...,2,False,True,D-inBC-AuxB-VIMA-80k,hso


## Show the success rate

In [2]:
def show_results(data, data_total):
    # Pivot the table
    grouped = data.groupby(['method', 'prompt_mode', 'level']).size().unstack(fill_value=0)
    total = data_total.groupby(['method', 'prompt_mode', 'level']).size().unstack(fill_value=0)
    
    # Create a new dataframe with the desired text format
    result = total.copy().astype(str)  # Copy the structure of table1
    for col in total.columns:
        for idx in total.index:
            if total.loc[idx, col] > 0:
                try:
                    result.loc[idx, col] = f"{grouped.loc[idx, col]} / {total.loc[idx, col]} ({grouped.loc[idx, col] / total.loc[idx, col] * 100:.1f}%)"
                except KeyError:
                    result.loc[idx, col] = f"0 / {total.loc[idx, col]} (0.0%)"
            else:
                result.loc[idx, col] = 'N/A'
    
    display(result)

show_results(df[df['success']], df)
print('Results for L4 are not valid because there is no rotation data when the end effector is a spatula.')

,level,L1,L2,L3,L4
method,prompt_mode,,,,
D-inBC-AuxB-VIMA-80k,hso,234 / 260 (90.0%),229 / 260 (88.1%),190 / 240 (79.2%),27 / 80 (33.8%)
D-inBC-AuxD-VIMA-80k,hso,218 / 260 (83.8%),229 / 260 (88.1%),189 / 240 (78.8%),25 / 80 (31.2%)
D-inBC-VIMA-80k,hso,227 / 260 (87.3%),222 / 260 (85.4%),197 / 240 (82.1%),20 / 80 (25.0%)


Results for L4 are not valid because there is no rotation data when the end effector is a spatula.
